In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data Preprocess

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

C:\Users\Keara\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Keara\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Keara\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Keara\anaconda3\lib\site-pac

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to C:\Users\Keara\tensorflow_datasets\mnist\1.0.0. Subsequent calls will reuse this data.


In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [4]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [5]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [7]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image  /= 225.
    return image, label

In [8]:
scaled_train_and_validation_data = mnist_train.map(scale)

In [9]:
test_data = mnist_test.map(scale)

In [10]:
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [11]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))

# Model 

### Outline the model

In [21]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
      
])

### Optimizer and the loss function

In [22]:
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics =['accuracy'])

### Training

In [23]:
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
540/540 - 5s - loss: 0.2639 - accuracy: 0.9211 - val_loss: 0.1313 - val_accuracy: 0.9622
Epoch 2/5
540/540 - 5s - loss: 0.1022 - accuracy: 0.9681 - val_loss: 0.0919 - val_accuracy: 0.9722
Epoch 3/5
540/540 - 5s - loss: 0.0662 - accuracy: 0.9793 - val_loss: 0.0715 - val_accuracy: 0.9787
Epoch 4/5
540/540 - 5s - loss: 0.0509 - accuracy: 0.9834 - val_loss: 0.0686 - val_accuracy: 0.9803
Epoch 5/5
540/540 - 5s - loss: 0.0427 - accuracy: 0.9869 - val_loss: 0.0545 - val_accuracy: 0.9840


### Test the model

In [24]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 20ms/step - loss: 0.0769 - accuracy: 0.9782


In [25]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.82%
